# This script simulates the path finding for a window-cleaning robot
## control Unit (PC)
### Inputs: 
- the angle of slope with respect to gravity's direction from inclinometer
- True/false from bumping sensor(micro switch)

### Output:
- the speed of two continuous tracks

## robot
### state
- angle,position and the velocity of two treads

### sensoring
- angle and T/F for 8 micro switches

## Assumption:
- The window is perfectly verticle. It has the form of rectangle
- the robot can be seen as a rigid body.
- the unit for length is cm

In [1]:
# The system is divided into 2 parts, the robot and the controlUnit (upper-level PC)
%load_ext autoreload
%autoreload 2
from pygame.math import Vector2
import numpy as np
class robot:
    def __init__(self, x, y,clinging_on, angle=0.0, length=40):
        self.clinging_on = clinging_on
        self.position = Vector2(x, y)
        self.wheel_velocity = np.array([0.0, 0.0]) # (left,right)
        self.angle = angle # car's direction with respect to  the gravity direction
        self.length = length
        self.center_mass_velocity = 0
        self.dist_ms_from_corner =0.2 # the distance between the corner and the micro switches is fixed
                                    # To simplift the task, we can consider that the ms is coincident with the corner
        self.micro_switch_pos, self.micro_switch_measure= self.micro_switch_update()
        self.inclinometer_resolution = 1 #To make the simulation more realistic, we set the resolution of the inclinometer as 
#                                               1 degree
        
    def micro_switch_update(self):
        micro_switch_pos = {"upleft":self.position+Vector2(-self.length/2,self.length/2).rotate(self.angle-180),
                                "upright":self.position+Vector2(self.length/2,self.length/2).rotate(self.angle-180),
                                "downleft":self.position+Vector2(-self.length/2,-self.length/2).rotate(self.angle-180),
                                "downright":self.position+Vector2(self.length/2,-self.length/2).rotate(self.angle-180)}
        micro_switch_measure = {"upleft":False,"upright":False,"downleft":False,"downright":False} 
        # in reality, we have two mss for each corner, the ms measure for a corner is true when one 
            #the ms is bumped into the window boundary
        
        for micro_switch in micro_switch_measure.keys():
            micro_switch_measure[micro_switch] = self.ms_inside_detection(micro_switch_pos[micro_switch],self.clinging_on)
            
        return [micro_switch_pos,micro_switch_measure]
    
    def update(self,dt):
        #position and angle update
        self.center_mass_velocity = self.wheel_velocity.mean()
        self.position = self.position + Vector2(self.center_mass_velocity*dt*(np.sin((180-self.angle)/180*np.pi)),
                                               self.center_mass_velocity*dt*(np.cos((180-self.angle)/180*np.pi)))
        if self.wheel_velocity[1]-self.wheel_velocity[0] ==0:
            angular_velocity = 0
        else:
            turning_radius = self.wheel_velocity[1]*self.length/(self.wheel_velocity[1]-self.wheel_velocity[0])
            angular_velocity = self.wheel_velocity[1]/turning_radius
        self.angle = self.angle+dt*angular_velocity/np.pi*180
        if self.angle>=360:
            self.angle=self.angle%360 #modulo operation
        while self.angle<=0:
            self.angle+=360
        
        #sensoring feedback
    def ms_inside_detection(self,single_micro_switch_pos,window):
        # when the sensor does not reach the boundary, the micro_switch is False
        if ((single_micro_switch_pos[0]<=window.left_boundary) or \
        (single_micro_switch_pos[0]>=window.right_boundary) or \
        (single_micro_switch_pos[1]<=window.down_boundary) or \
        (single_micro_switch_pos[1]>=window.up_boundary)):
            return True
        else:
            return False
    
    
    def action(self, direction, radius, velocity):
        # the output of an action is the velocity of two treads. If the radius = 0, then the robot is spining. 
        # The radius = 'infty', the robot move straightly. The velocity is the speed of the mass center.
        if radius == 'infty':
            self.wheel_velocity = np.array([velocity,velocity])
        elif direction == 'left':
            if radius !=0:
                self.wheel_velocity = np.array([velocity*(radius-self.length/2)/radius,velocity*(radius+self.length/2)/radius])
            else:
                self.wheel_velocity = np.array([-velocity,velocity])
        elif direction == 'right':
            if radius !=0:
                self.wheel_velocity = np.array([velocity*(radius+self.length/2)/radius,velocity*(radius-self.length/2)/radius])
            else:
                self.wheel_velocity = np.array([velocity,-velocity])
        else:
            print ('unidentified direction')
class window():
    def __init__(self,height=500,width=500):
        self.height = height
        self.width = width
        self.left_boundary = 0
        self.down_boundary = 0
        self.right_boundary = self.width
        self.up_boundary = self.height    
# class controlUnit:
#     def __init__(self, sensor_measure):

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [9]:
# Visualization
import pygame
from IPython.core.debugger import set_trace
class Game:
    def __init__(self,robot_to_use):
        global WHITE
        WHITE = (255,255,255)
        global BLUE 
        BLUE= (135,206,250)
        global RED
        RED = (255,0,0)
        global GREEN
        GREEN = (0, 255, 0)
        pygame.init()
        pygame.display.set_caption("window-cleaning robot simulation")
        width = 1280
        height = 720
        self.screen = pygame.display.set_mode((width, height))
        self.clock = pygame.time.Clock()
        self.ticks = 60
        self.exit = False
        self.robot_to_use = robot_to_use
        self.window_to_clean = self.robot_to_use.clinging_on
        self.state_orientation_init = [False]*5 # The orientation has 4 states: turning to left, hitting the boundary
                                                        # turning towards to the lower boundary, hittng it
        self.sending_same_v_times = 20
        self.counter_sending_same_v_times = 0
        self.cleaning_steering_duration = 100
        self.counter_cleaning_steering_duration =0
        self.next_cleaning_step = 'up'
        
        
        self.reference_point = [10,10] # the reference point for drawing, we use it only for esthetic purpose
        
    def draw_window(self):
        
        window_surface = pygame.Surface((self.window_to_clean.width,self.window_to_clean.height))
        window_surface.fill(WHITE)
        window_surface.set_colorkey((0,0,0))
        rect=window_surface.get_rect()
        self.screen.blit(window_surface, rect)
        
        
    def draw_robot (self):
        
        robot_surface = pygame.Surface((self.robot_to_use.length,self.robot_to_use.length))
        robot_surface.fill(BLUE)
        robot_surface.set_colorkey((0,0,0)) 

        
        #draw indicative arrow
        pygame.draw.polygon(robot_surface,RED, (Vector2(self.robot_to_use.length/2, self.robot_to_use.length/2),
                                                Vector2(self.robot_to_use.length/2,0), Vector2(self.robot_to_use.length/4, 
                                               self.robot_to_use.length/4), Vector2(3*self.robot_to_use.length/4, 
                                                self.robot_to_use.length/4), Vector2(self.robot_to_use.length/2,0)))
        #rotation
        robot_surface_rotated = pygame.transform.rotate(robot_surface, self.robot_to_use.angle-180)
        # bliting
        self.screen.blit(robot_surface_rotated,(self.robot_to_use.position[0]-self.robot_to_use.length/2,
                                        self.window_to_clean.height - self.robot_to_use.position[1]-self.robot_to_use.length/2))
        
        return True
    
    def draw_finish (self):
        font = pygame.font.Font('freesansbold.ttf', 32) 
        text = font.render('Cleaning finished', True, GREEN, BLUE) 
        textRect = text.get_rect()  
        textRect.center = (self.window_to_clean.width/ 2,  self.window_to_clean.height/ 2) 
        self.screen.blit(text,textRect)
    
    # the orientation module whose purpose is to place the robot in the lower-left corner 
    def orientation_init(self):
        # The robot first turns to the left, then walk straightly until it hits the boundary. It steps back and turns
        # to the front direction and walk reversely to the lower left corner
        self.robot_to_use.micro_switch_pos, self.robot_to_use.micro_switch_measure = self.robot_to_use.micro_switch_update()
        if self.state_orientation_init == [True]*5:
            return True
        if self.state_orientation_init[0] == False:
            if 90<=self.robot_to_use.angle <270:
                self.robot_to_use.action('left',0,1)
            else:
                self.robot_to_use.action('right',0,1)
            if self.robot_to_use.angle >270-self.robot_to_use.inclinometer_resolution and \
            self.robot_to_use.angle <270+self.robot_to_use.inclinometer_resolution:
                self.state_orientation_init[0]=True
                
                # second stage
        if self.state_orientation_init[0]==True and self.state_orientation_init[1] == False:
            if self.robot_to_use.micro_switch_measure['upleft'] == True or \
            self.robot_to_use.micro_switch_measure['upright'] == True:
                self.state_orientation_init[1] = True
                self.robot_to_use.action('left','infty',0)
#                 set_trace()
            else: 
                self.robot_to_use.action('left','infty',1)
        
        if (self.state_orientation_init[0:2]==[True,True] and self.state_orientation_init[2] == False):
            if self.counter_sending_same_v_times<=self.sending_same_v_times:
                self.robot_to_use.action('left','infty',-1)
                self.counter_sending_same_v_times+=1
            else:
                self.robot_to_use.action('left','infty',0)
                self.counter_sending_same_v_times = 0
                self.state_orientation_init[2] = True
        if (self.state_orientation_init[0:3]==[True]*3 and self.state_orientation_init[3] == False):
            self.robot_to_use.action('right',0,1)
#             set_trace()
            if self.robot_to_use.angle >180-self.robot_to_use.inclinometer_resolution and \
            self.robot_to_use.angle <180+self.robot_to_use.inclinometer_resolution:
                self.state_orientation_init[3]=True
                self.robot_to_use.action('left','infty',0)
        if (self.state_orientation_init[0:4]==[True]*4 and self.state_orientation_init[4] == False):
            if self.robot_to_use.micro_switch_measure['downleft'] == True or \
            self.robot_to_use.micro_switch_measure['downright'] == True:
                self.robot_to_use.action('left','infty',0)
                self.state_orientation_init[4] = True
            else:
                self.robot_to_use.action('left','infty',-1)              
        return False
    
    def cleaning(self):
        # the cleaning consists of 4 steps: going upward, turning reversily to right, resetting the robot at gravity direction
        # going backward
        self.robot_to_use.micro_switch_pos, self.robot_to_use.micro_switch_measure = self.robot_to_use.micro_switch_update()
        if self.robot_to_use.micro_switch_measure['upright'] == True and \
                self.robot_to_use.micro_switch_measure['downright'] == True:
            return True
        if self.next_cleaning_step == 'up':
            if self.robot_to_use.micro_switch_measure['upleft'] == False or \
                self.robot_to_use.micro_switch_measure['upright'] == False:
                self.robot_to_use.action('left','infty',1)
            else:
                self.next_cleaning_step = 'turn1'
                self.robot_to_use.action('left','infty',0)
        if self.next_cleaning_step == 'turn1':
            if self.counter_cleaning_steering_duration <= self.cleaning_steering_duration:
                self.robot_to_use.action('right',10,-0.8) # the speed is set high to spped up the testing
                self.counter_cleaning_steering_duration+=1
            else:
                self.next_cleaning_step = 'turn2'
                self.robot_to_use.action('left','infty',0)
                self.counter_cleaning_steering_duration=0
        if self.next_cleaning_step == 'turn2':
            if self.robot_to_use.angle<=180+self.robot_to_use.inclinometer_resolution and \
            self.robot_to_use.angle>= 180 -self.robot_to_use.inclinometer_resolution:
                self.next_cleaning_step = 'down'
                self.robot_to_use.action('left','infty',0)
            else:
#                 set_trace()
                self.robot_to_use.action('left',40,-0.8)
        if self.next_cleaning_step == 'down':
            if self.robot_to_use.micro_switch_measure['downleft'] == False or \
                self.robot_to_use.micro_switch_measure['downright'] == False:
                self.robot_to_use.action('left','infty',-1)
            else:
                self.next_cleaning_step = 'up'
                self.robot_to_use.action('left','infty',0)
        return False
            
        

    def run(self):
        while not self.exit:
            dt = self.clock.get_time() / 1000

            # Event queue
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.exit = True

            # User input
            pressed = pygame.key.get_pressed()

            # Logic
            # Car logic goes here

            # Drawing
            self.screen.fill((0, 0, 0))
            self.draw_window()
            self.draw_robot()
            
            # update
            orientation_finished = self.orientation_init()     
            if orientation_finished == True:
                cleaning_finished = self.cleaning()
                if cleaning_finished ==True:
                    self.draw_finish()
                # path planing goes here
            self.robot_to_use.update(0.1)
            pygame.display.flip()

            self.clock.tick(self.ticks)
        pygame.quit()


if __name__ == '__main__':
    window_to_clean = window(height = 80, width =200)
    # initialization : the robot was clinging on the window with pre-defined position
    robot_to_use = robot(40,40,window_to_clean,250)
    game = Game(robot_to_use)
    game.run()
    
# minor comments for pygame
# pygame.display.flip() updates the entire Surface on the display.
# pygame.display.update() updates the entire Surface, 
# only if no argument is passed. You can pass rectangle(s) instead to update only that portion of the Surface, 
# instead of the entire surface.
# The declaration for this function is

In [11]:
#Test cell

#prerequisite reloading before the excution of the following cells
state_of_orientation_init




NameError: name 'state_of_orientation_init' is not defined